In [1]:
import math
import numpy as np
import planet_sampler_pop as ps
import pandas as pd

## Importing the data

In [2]:
weiss_data = pd.read_csv("weiss_tables.csv")
kam_planets = ps.PlanetSample("emile_testing_AG13_bright5_baseline.txt", dataset = "SAG")
life_planets = ps.PlanetSample("emile_testing_AG13_bright5_weiss.txt", dataset = "SAG")

## Defining some functions

In [3]:
def stability_criterion(M_star, a_in, a_out, m_in, m_out,  e_in, e_out, threshold):
    m_in = m_in * 5.972E24
    m_out = m_out * 5.972E24 #getting the planet masses in kg
    M_star = M_star * 1.989E30 # getting the star mass in kg
    hill_radius = (a_in+a_out)/2 * ((m_in + m_out)/(3*M_star))**(1/3)
    delta = (a_out*(1-e_out) - a_in*(1+e_in))/hill_radius
    return delta > threshold

## Getting the results for the original sample of Kam&Quanz

In [4]:
star_sample = np.arange(0,326,1)
count_true = 0
count_false = 0
count_true_syst, count_false_syst = 0,0
for n in np.unique(life_planets.nMC):
    for star in star_sample:
        temp = 0
        syst = np.logical_and(kam_planets.snumber == star, kam_planets.nMC == n)
        if(np.count_nonzero(syst) > 1):
            a_planets = kam_planets.a[syst]
            e_planets = kam_planets.ecc[syst]
            M_star = kam_planets.Ms[syst]
            mass_planets = kam_planets.Mp[syst]
            a_planets = a_planets[np.argsort(a_planets)]
            e_planets = e_planets[np.argsort(a_planets)]
            mass_planets = mass_planets[np.argsort(a_planets)]
            for i in range(len(a_planets) -1 ):#We loop over all the planet pairs
                if(stability_criterion(M_star[0], a_planets[i], a_planets[i+1], mass_planets[i], mass_planets[i+1], e_planets[i], e_planets[i+1], 8)):
                    count_true += 1
                else:#If at least one pair is unstable
                    temp = 1 #We discard the system
                    count_false += 1
            if(temp == 0):
                count_true_syst +=1 
            elif(temp == 1):
                count_false_syst +=1
                
fraction_true = math.floor(count_true /(count_true + count_false)*100)
print("They were " + str(count_true) + " pairs that passed the test, which represent " + str(fraction_true) + " % of the total number of pairs (" + str(count_true + count_false) + ")")
fraction_true_syst = math.floor(count_true_syst /(count_true_syst + count_false_syst)*100)
print("They were " + str(count_true_syst) + " syst that passed the test, which represent " + str(fraction_true_syst) + " % of the total number of syst (" + str(count_true_syst + count_false_syst) + ")")

They were 2451 pairs that passed the test, which represent 73 % of the total number of pairs (3318)
They were 731 syst that passed the test, which represent 56 % of the total number of syst (1287)


## Getting the results for my update version

In [5]:
star_sample = np.arange(0,326,1)
count_true = 0
count_true_syst, count_false_syst = 0,0
count_false = 0
for n in np.unique(life_planets.nMC):
    for star in star_sample:# Loop on each system
        temp = 0
        syst = np.logical_and(life_planets.snumber == star, life_planets.nMC == n)
        if(np.count_nonzero(syst) > 1):#If there is more than 1 planet in the system
            a_planets = life_planets.a[syst]
            e_planets = life_planets.ecc[syst]
            M_star = life_planets.Ms[syst]
            mass_planets = life_planets.Mp[syst]
            a_planets = a_planets[np.argsort(a_planets)]
            e_planets = e_planets[np.argsort(a_planets)]
            mass_planets = mass_planets[np.argsort(a_planets)]
            for i in range(len(a_planets) -1 ):#We loop over all the planet pairs
                if(stability_criterion(M_star[0], a_planets[i], a_planets[i+1], mass_planets[i], mass_planets[i+1], e_planets[i], e_planets[i+1], 8)):
                    count_true += 1
                else:#If at least one pair is unstable
                    temp = 1 #We discard the system
                    count_false += 1
            if(temp == 0):
                count_true_syst +=1 
            elif(temp == 1):
                count_false_syst +=1
            
fraction_true = math.floor(count_true /(count_true + count_false)*100)
print("They were " + str(count_true) + " pairs that passed the test, which represent " + str(fraction_true) + " % of the total number of pairs (" + str(count_true + count_false) + ")")
fraction_true_syst = math.floor(count_true_syst /(count_true_syst + count_false_syst)*100)
print("They were " + str(count_true_syst) + " syst that passed the test, which represent " + str(fraction_true_syst) + " % of the total number of syst (" + str(count_true_syst + count_false_syst) + ")")

They were 2646 pairs that passed the test, which represent 100 % of the total number of pairs (2646)
They were 1177 syst that passed the test, which represent 100 % of the total number of syst (1177)


In [6]:
st = 325
for i in range(5):
    if(np.count_nonzero(np.logical_and(life_planets.nMC == i, life_planets.snumber == st)) > 1):
        temp = np.logical_and(life_planets.nMC == i, life_planets.snumber == st)
        print(life_planets.period[temp])
        print(life_planets.a[temp])


[ 3.38363  7.02912  8.51359 12.95181]
[0.04875 0.07937 0.09019 0.1193 ]
[11.96866 27.67776 44.79097]
[0.11318 0.19792 0.27281]
[119.90437 214.71682 435.79049 766.69374]
[0.52597 0.77562 1.24334 1.81198]
[12.62148 20.18522]
[0.11726 0.16036]
[ 4.38026  7.44943 17.77734]
[0.05791 0.08251 0.14734]


## Getting the results for Weiss et al dataset

In [8]:
count_true = 0
count_false = 0
for star in weiss_data.host_star.unique():
    syst = weiss_data[weiss_data.host_star == star]
    syst_order = np.argsort(np.array(syst.planet_period))
    mutual_hills = np.array(syst.Delta)[syst_order]
    for i in range(len(syst)- 1):
        if(mutual_hills[i+1] > 8):
            count_true += 1
        else:
            count_false += 1
fraction_true = math.floor(count_true /(count_true + count_false)*100)
print("They were " + str(count_true) + " pairs that passed the test, which represent " + str(fraction_true) + " % of the total number of pairs (" + str(count_true + count_false) + ")")

They were 543 pairs that passed the test, which represent 98 % of the total number of pairs (554)


In [9]:
def a(Ms, Porb):
		"""
		Computes planet semi-major axis (in astronomical units)
		"""
		
		# Define constants
		G = 6.67408E-11
		Msun = 1.989E+30
		au = 149597870700.
		
		# Return planet semi-major axis
		return ((G*Ms*Msun*(Porb*86400.)**2.)/(4.*np.pi**2.))**(1./3.)/au